In [1]:
!pip install asteval attrs attrdict

In [1]:
'''
Demos written and developed by: Andrew E. Oriani 2020 EMAIL: oriani@uchicago.edu

'''
from PyInventor import *
import numpy as np
from numpy import pi, cos, sin, sqrt, log
import math
import glob
import time
import datetime

#Import plotting libraries for later
import matplotlib
from matplotlib import pyplot as plt

from matplotlib import cm
import matplotlib as mpl
from matplotlib.colors import BoundaryNorm 
from matplotlib.ticker import MaxNLocator

import pandas as pd

import pyEPR
from pyEPR import ansys as HFSS
from pyEPR.core_distributed_analysis import CalcObject

import scipy
from scipy import constants as const

from pint import UnitRegistry
ureg = UnitRegistry()
Q = ureg.Quantity
import os
import re as regex

# Field integrals code
import sys
sys.path.append('../python/src')
import field_integrals as fint

In [3]:
'''
Some code for frequency estimation
'''

class unique_element:
    def __init__(self,value,occurrences):
        self.value = value
        self.occurrences = occurrences
        
def perm_unique(elements):
    eset=set(elements)
    listunique = [unique_element(i,elements.count(i)) for i in eset]
    u=len(elements)
    return perm_unique_helper(listunique,[0]*u,u-1)

def perm_unique_helper(listunique,result_list,d):
    if d < 0:
        yield tuple(result_list)
    else:
        for i in listunique:
            if i.occurrences > 0:
                result_list[d]=i.value
                i.occurrences-=1
                for g in  perm_unique_helper(listunique,result_list,d-1):
                    yield g
                i.occurrences+=1

def TE_mode_sort(mode_num=1):
    '''
    This calculates allowable nml values for the TE modes of the resonator. Uses above functions
    '''
    
    permutes=[[1,0,1],[1,0,2],[1,0,3],[1,0,4],[1,0,5],[2,0,2],[2,0,3],[2,0,4],[2,0,5],[3,0,3],[3,0,4],[3,0,5]]
    nml=[]
    for i in range(len(permutes)):
        perms=np.array(list(perm_unique(permutes[i])))
        for vals in perms:
            if vals[0]!=0 and vals[1]==0 and vals[2]!=0:
                nml.append(list(vals))
            else:
                pass
    nml=np.array(nml)
    i_sort=np.argsort(nml[:,0])
    return nml[i_sort][0:mode_num]


def freq_rect(a,b,c, modes=1, unit='metric'):
    '''
    Calculate the lowest n-mode frequencies for a rectangular cavity of a,b,c dimensions where b is the smallest dim
    '''
    if unit=='metric':
        a=a/1000.
        b=b/1000.
        c=c/1000.
    elif unit=='imperial':
        a=a*.0254
        b=b*.0254
        c=c*.0254
    else:
        raise Exception('ERROR: Unit must be metric or imperial')
    
    eps=const.epsilon_0
    mu=const.mu_0
    c_speed=const.c
    nml_array=TE_mode_sort(10)
    f=[]

    for vals in nml_array:
        n=vals[0]
        m=vals[1]
        l=vals[2]
        f.append(((c_speed)/(2*pi))*sqrt((n*pi/a)**2+(m*pi/b)**2+(l*pi/c)**2))
    f=np.array(f)
    f_sort=np.argsort(f)
    return f[f_sort][0:modes], nml_array[f_sort]

def get_c_dim(f,a,b, unit='metric'):
    '''
    Estimates the second long dimension based on a target frequency and the other two dimensions. Works only for the 
    fundamental mode
    '''
    brentq=scipy.optimize.brentq
    f_intercept=lambda c:freq_rect(a,b,c,1, unit)[0][0]-f
    c=brentq(f_intercept, min([10*a, b]), max([10*a, b]))

    return c

def round_dim(dim, frac):
    val=frac*math.ceil(dim/frac)+frac
    if (val-dim)/2<frac:
        val=val+.25
    else:
        pass
    return val

def check_path(path):
    check=glob.glob(path)
    if check==[]:
        return False
    else:
        return True

def cap_sigma(cmat, units='fF'):
    cmat=Q(cmat, units).to('F').magnitude
    c11=abs(cmat[0,0])-abs(cmat[0,1])
    c22=abs(cmat[1,1])-abs(cmat[1,0])
    c12=abs(cmat[0,1])/2+abs(cmat[1,0])/2
    return (c12+(c11*c22)/(c11+c22))

def C_to_Ec(cmat, cap_units='fF'):
    csig=cap_sigma(cmat, cap_units)
    print (csig)
    return const.e**2/(2*(csig))/const.h

def C_to_Ec(cmat, cap_units='fF'):
    csig=cap_sigma(cmat, cap_units)
    return const.e**2/(2*(csig))/const.h

def Ec_to_C(Ec):
    Ec = Ec*const.h
    return const.e**2/(2*Ec)

def L_from_nu_and_C(nu,C):
    return (1/nu**2/4/np.pi**2/C)

In [4]:
'''
Some code for frequency estimation
'''

class unique_element:
    def __init__(self,value,occurrences):
        self.value = value
        self.occurrences = occurrences
        
def perm_unique(elements):
    eset=set(elements)
    listunique = [unique_element(i,elements.count(i)) for i in eset]
    u=len(elements)
    return perm_unique_helper(listunique,[0]*u,u-1)

def perm_unique_helper(listunique,result_list,d):
    if d < 0:
        yield tuple(result_list)
    else:
        for i in listunique:
            if i.occurrences > 0:
                result_list[d]=i.value
                i.occurrences-=1
                for g in  perm_unique_helper(listunique,result_list,d-1):
                    yield g
                i.occurrences+=1

def TE_mode_sort(mode_num=1):
    '''
    This calculates allowable nml values for the TE modes of the resonator. Uses above functions
    '''
    
    permutes=[[1,0,1],[1,0,2],[1,0,3],[1,0,4],[1,0,5],[2,0,2],[2,0,3],[2,0,4],[2,0,5],[3,0,3],[3,0,4],[3,0,5]]
    nml=[]
    for i in range(len(permutes)):
        perms=np.array(list(perm_unique(permutes[i])))
        for vals in perms:
            if vals[0]!=0 and vals[1]==0 and vals[2]!=0:
                nml.append(list(vals))
            else:
                pass
    nml=np.array(nml)
    i_sort=np.argsort(nml[:,0])
    return nml[i_sort][0:mode_num]


def freq_rect(a,b,c, modes=1, unit='metric'):
    '''
    Calculate the lowest n-mode frequencies for a rectangular cavity of a,b,c dimensions where b is the smallest dim
    '''
    if unit=='metric':
        a=a/1000.
        b=b/1000.
        c=c/1000.
    elif unit=='imperial':
        a=a*.0254
        b=b*.0254
        c=c*.0254
    else:
        raise Exception('ERROR: Unit must be metric or imperial')
    
    eps=const.epsilon_0
    mu=const.mu_0
    c_speed=const.c
    nml_array=TE_mode_sort(10)
    f=[]

    for vals in nml_array:
        n=vals[0]
        m=vals[1]
        l=vals[2]
        f.append(((c_speed)/(2*pi))*sqrt((n*pi/a)**2+(m*pi/b)**2+(l*pi/c)**2))
    f=np.array(f)
    f_sort=np.argsort(f)
    return f[f_sort][0:modes], nml_array[f_sort]

def get_c_dim(f,a,b, unit='metric'):
    '''
    Estimates the second long dimension based on a target frequency and the other two dimensions. Works only for the 
    fundamental mode
    '''
    brentq=scipy.optimize.brentq
    f_intercept=lambda c:freq_rect(a,b,c,1, unit)[0][0]-f
    c=brentq(f_intercept, min([10*a, b]), max([10*a, b]))

    return c

def round_dim(dim, frac):
    val=frac*math.ceil(dim/frac)+frac
    if (val-dim)/2<frac:
        val=val+.25
    else:
        pass
    return val

def check_path(path):
    check=glob.glob(path)
    if check==[]:
        return False
    else:
        return True

def cap_sigma(cmat, units='fF'):
    cmat=Q(cmat, units).to('F').magnitude
    c11=abs(cmat[0,0])-abs(cmat[0,1])
    c22=abs(cmat[1,1])-abs(cmat[1,0])
    c12=abs(cmat[0,1])/2+abs(cmat[1,0])/2
    return (c12+(c11*c22)/(c11+c22))

def C_to_Ec(cmat, cap_units='fF'):
    csig=cap_sigma(cmat, cap_units)
    print (csig)
    return const.e**2/(2*(csig))/const.h

def C_to_Ec(cmat, cap_units='fF'):
    csig=cap_sigma(cmat, cap_units)
    return const.e**2/(2*(csig))/const.h

def Ec_to_C(Ec):
    Ec = Ec*const.h
    return const.e**2/(2*Ec)

def L_from_nu_and_C(nu,C):
    return (1/nu**2/4/np.pi**2/C)

# PyInventor and cavity design:
The following shows a basic example of how to setup and compile an Inventor design using the PyInventor designs. To learn more about PyInventor and see various examples of how PyInventor can be used for cavity design please refer to: https://github.com/AndrewOriani/PyInventor and the tutorial and demo notebooks contained within.


In [5]:
def single_pan_pipe(target_freq, hole_spacing=0.2375,
                   hole_dia=0.1875, H_stock=2.25,
                    evan_length=0.5,
                   units='imperial',
                   fname='single_pan_pipe.ipt'):
    """
    Creates a single mode pan-pipe cavity with inventor
    """
    # Start at the origin
    stock_origin=(0,0)

    #calculate cavity height allowed by stock
    H_cav=H_stock-2*evan_length

    #calculate what you want the length of t he cavity to be for  the proper fundamental frequency
    L_cav=get_c_dim(target_freq, H_cav, hole_dia, units)

    #set the maximum number of holes, call them top
    num_holes_top=round((L_cav-hole_dia)/hole_spacing)+1

    #number of bottom holes
    num_holes_bot=num_holes_top-1

    #recalculate frequency now that holes are set
    eff_cav_L=(num_holes_top-1)*hole_spacing+hole_dia
    H_cav=get_c_dim(target_freq, eff_cav_L, hole_dia, units)

    # calculate stock length for the given cavity length needed
    # rounds length to nearest designated fraction
    L_stock=round_dim(L_cav, .25)
    W_stock=round_dim(hole_dia, .125)

    #start inventor and close all inventor objects
    com_obj().close_all_parts()

    #set it to overwrite file every time the part is instantiated (REMEMBER TO PUT SAVE AT THE BOTTOM!)
    overwrite=True

    # path='Z:\\Andrew Oriani\\CQED 3D Resonators\\Multimode Manipulate Cavity\\3D MM V3'

    #for demo set the path to path='' to save in current working directory
    path=os.getcwd() + '\\Inventor Files\\'

    #setup part 
    part=iPart(path=path, prefix=fname, units=units, overwrite=overwrite)

    #set view as shaded with edges
    part.set_visual_style(shaded=True, edges=True, hidden_edges=True)

    #set starting workplane
    lower_wp=part.add_workplane(plane='xy')

    #attach a sketch to the workplane
    sketch_1=part.new_sketch(lower_wp)

    #set upper workplane
    upper_wp=part.add_workplane(plane='xy', offset=H_stock)

    #attach a sketch to the upper workplane
    sketch_2=part.new_sketch(upper_wp)

    #create stock base associated with sketch 1. The structuire object keeps track of position
    stock_base=structure(part, sketch_1, start=stock_origin)

    stock_base.add_line(L_stock, 180)
    stock_base.add_line(W_stock, 90)
    stock_base.add_line(L_stock, 0)
    stock_base.add_line(W_stock, 270)

    #structure identifier
    base_shape=stock_base.draw_path(close_path=True)

    #extrude base shape. It is important to set the object collection (obj_collection) to the specific polygon shape we created
    base_extrusion=part.extrude(sketch_1, thickness=H_stock, obj_collection=base_shape, direction='positive', operation='join')

    #fits whole cavity in frame
    part.view.Fit()

    #hole starting coordinate
    hole_start=(-(L_stock-L_cav+hole_dia)/2, W_stock/2)

    #structure identifier
    cav_holes_top=structure(part, sketch_1, start=hole_start)
    cav_holes_bot=structure(part, sketch_2, start=(hole_start[0]-hole_spacing/2, hole_start[1]))

    #set total hole distance for top holes
    hole_dist_top=hole_spacing*(num_holes_top-1)
    hole_dist_bot=hole_spacing*(num_holes_bot-1)

    #start drawing points for top holes
    cav_holes_top.add_point_line(distance=hole_dist_top , direction=180, num_points=num_holes_top)   
    top_holes_pos=cav_holes_top.get_pts()[0]

    #create top holes by just putting in the list of positions. It will automatically make an object collection
    hole_depth=H_cav+evan_length
    part.new_hole(sketch=sketch_1, pos=top_holes_pos, dia=hole_dia, depth=hole_depth, direction='negative', FlatBottom=False, BottomTipAngle=None)

    #start drawing holes for bottom points
    cav_holes_bot.add_point_line(distance=hole_dist_bot , direction=180, num_points=num_holes_bot)   
    bot_holes_pos=cav_holes_bot.get_pts()[0]

    #create bottom holes
    part.new_hole(sketch=sketch_2, pos=bot_holes_pos, dia=hole_dia, depth=hole_depth, direction='positive', FlatBottom=False, BottomTipAngle=None)

    part.view.GoHome()

    #Save document 
    part.save()

    #save copy as stp file for export to HFSS
    path=part.save_copy_as(copy_name=fname.split('.')[0]+'.stp' );

    print('Estimated TE_101 mode frequency: %.3e'%target_freq)
    
    return L_stock, W_stock, H_stock, bot_holes_pos, top_holes_pos

# Eigenmode:

<img src="Pictures/Cavity Eigenmode.png" alt="drawing" width="400">

Determines first 5 eigenmode frequencies in absence of any qubit. This demonstration also calculates S_e and S_m (electric and magnetic field participation of the cavity modes) for the first (TE_101) mode.

In [6]:
'''
HFSS eigenmode simulation Creation:

This sets up a standard eigenmode simulation without the qubit, just the bare cavity created above. It calculates
the mode frequencies, loss (with boundary impedances set) and the electric and magnetic surface participation 
ratios (S_e, S_m)

'''
path = 'Z:\\tcav\\ipynbs\\Inventor Files\\'
project_name='Flute_Test_EM'
design_name='Pan_pipe_Test'

stp_exists = glob.glob(path+'flute_cavity_6p375_GHz.stp')
print(f'flute_cavity_6p375_GHz.stp exists: {stp_exists}')
stp_exists = glob.glob(path+'flute_cavity_7p375_GHz.stp')
print(f'flute_cavity_7p475_GHz.stp exists: {stp_exists}')

# Project and HFSS design names for the bare waveguide model
dd = datetime.datetime.today().strftime('%y%m%d')
solution_type = 'eigenmode'

# solution_type = 'driven_modal'
#dd = '230120'
#dd = '230121'
overwrite=False
dd = '230829'
yep = 'planar_pads_l_coupled_three_cavities_degen_eigen_hfss_design_230120'
ref_name = 'planar_pads_l_coupled_three_cavities_degen_eigen'
# ref_name = 'planar_pads_l_coupled_three_cavities_degen'
project_name=f'{ref_name}_{dd}'
design_name=f'{ref_name}_hfss_design_{dd}'
# design_name=f'{ref_name}_hfss_design_no_jj_230121'


#use file location path:
HFSS_path=os.getcwd()

full_path=HFSS_path+'\\HFSS\\'+project_name+'.aedt'

HFSS_app=HFSS.HfssApp()
HFSS_desktop=HFSS_app.get_app_desktop()

project=HFSS_desktop.open_project(full_path)

if project==None:
    project=HFSS_desktop.new_project()
    project.save(full_path)
    
project.save(full_path)
project.make_active()
    
if design_name in project.get_design_names():
    if overwrite==True:
        project.delete_design(design_name)
        project.save()
        
        # Setup a driven modal design type
        if solution_type == 'driven_modal':
            EM_design=project.new_dm_design(design_name)
        elif solution_type == 'eigenmode':
            EM_design=project.new_em_design(design_name)
        else:
            raise ValueError(f'Solution type ({solution_type}) not recognized.')
    else:
        EM_design=project.get_design(design_name)
        
else:
    if solution_type == 'driven_modal':
        EM_design=project.new_dm_design(design_name)
    elif solution_type == 'eigenmode':
        EM_design=project.new_em_design(design_name)
    else:
        raise ValueError(f'Solution type ({solution_type}) not recognized.')
        
EM_design.make_active()
model=HFSS.HfssModeler(EM_design)

project=HFSS_desktop.open_project(full_path)

if project==None:
    project=HFSS_desktop.new_project()
    project.save(full_path)
    
project.make_active()
    
if design_name in project.get_design_names():
    if overwrite==True:
        project.delete_design(design_name)
        project.save()
        EM_design=project.new_em_design(design_name)
    else:
        EM_design=project.get_design(design_name)
        
else:
    EM_design=project.new_em_design(design_name)
        
EM_design.make_active()
model=HFSS.HfssModeler(EM_design)

flute_cavity_6p375_GHz.stp exists: ['Z:\\tcav\\ipynbs\\Inventor Files\\flute_cavity_6p375_GHz.stp']
flute_cavity_7p475_GHz.stp exists: ['Z:\\tcav\\ipynbs\\Inventor Files\\flute_cavity_7p375_GHz.stp']
project_name: planar_pads_l_coupled_three_cavities_degen_eigen_230829
path: Z:\tcav\ipynbs\HFSS\planar_pads_l_coupled_three_cavities_degen_eigen_230829.aedt
project_name: planar_pads_l_coupled_three_cavities_degen_eigen_230829
path: Z:\tcav\ipynbs\HFSS\planar_pads_l_coupled_three_cavities_degen_eigen_230829.aedt


In [6]:
if overwrite:
    # Two calls to pan-pipe design
    L_stock, W_stock, H_stock, bot_holes_pos, top_holes_pos \
    = single_pan_pipe(target_freq=6.375e9, fname='flute_cavity_6p375_GHz.ipt')
    L_stock2, W_stock2, H_stock2, bot_holes_pos2, top_holes_pos2 \
    = single_pan_pipe(target_freq=7.375e9, fname='flute_cavity_7p375_GHz.ipt')

    # Stock subtraction for cavity 1
    model.import_3D_obj(path+'flute_cavity_6p375_GHz.stp')
    Stock_L=EM_design.set_variable('Stock_L', '%.3fin'%(-L_stock))
    Stock_W=EM_design.set_variable('Stock_W', '%.3fin'%(W_stock))
    Stock_H=EM_design.set_variable('Stock_H', '%.3fin'%(H_stock))

    cav_dims=[Stock_L, Stock_W, Stock_H]

    box=model.draw_box_corner([0,0,0], cav_dims, name='cavity1')
    objs=model.get_object_names()
    obj_name=model.subtract(blank_name=objs[1], tool_names=[objs[0]])

    # Stock subtraction for cavity 2
    # model.import_3D_obj(path+'flute_cavity_6p575_GHz.stp')
    model.import_3D_obj(path+'flute_cavity_7p375_GHz.stp')
    Stock_L2=EM_design.set_variable('Stock_L2', '%.3fin'%(-L_stock2))
    Stock_W2=EM_design.set_variable('Stock_W2', '%.3fin'%(W_stock2))
    Stock_H2=EM_design.set_variable('Stock_H2', '%.3fin'%(H_stock2))

    cav_dims2=[Stock_L2, Stock_W2, Stock_H2]

    box2=model.draw_box_corner([0,0,0], cav_dims2, name='cavity2')
    obj_name2=model.subtract(blank_name='cavity2', tool_names=['Solid1_1'])

    # Cavity 1 impedance boundary conditions
    bot_hole_fid=[]
    for locs in bot_holes_pos:
        loc=['%.4fin'%locs[0], '%.4fin'%locs[1], Stock_H]
        fid=model.get_face_id_by_pos(obj_name, pos=loc)
        bot_hole_fid.append(fid)

    top_hole_fid=[]
    for locs in top_holes_pos:
        loc=['%.4fin'%locs[0], '%.4fin'%locs[1], '%.4fin'%0]
        fid=model.get_face_id_by_pos(obj_name, pos=loc)
        top_hole_fid.append(fid)

    top_imped=model.assign_impedance(377, 0, obj_name, 
                                     top_hole_fid, name='Top_Imped')
    bot_imped=model.assign_impedance(377, 0, obj_name,
                                     bot_hole_fid, name='Bot_Imped')

    # Cavity 2 impedance boundary conditions
    bot_hole_fid=[]
    for locs in bot_holes_pos2:
        loc=['%.4fin'%locs[0], '%.4fin'%locs[1], Stock_H]
        fid=model.get_face_id_by_pos(obj_name2, pos=loc)
        bot_hole_fid.append(fid)

    top_hole_fid=[]
    for locs in top_holes_pos2:
        loc=['%.4fin'%locs[0], '%.4fin'%locs[1], '%.4fin'%0]
        fid=model.get_face_id_by_pos(obj_name2, pos=loc)
        top_hole_fid.append(fid)

    top_imped=model.assign_impedance(377, 0, obj_name, 
                                     top_hole_fid, name='Top_Imped2')
    bot_imped=model.assign_impedance(377, 0, obj_name,
                                     bot_hole_fid, name='Bot_Imped2')

    # Move cavity #2 along a vector
    intomm = 25.4
    x_offset_mm = 15.
    xoffset = EM_design.set_variable('x_offset', f'{x_offset_mm:.3f}mm')
    xcorr_val = 7.904534763+1.0923377
    xcorr = EM_design.set_variable('xoff_corr', f'{xcorr_val:.10f}mm')
    xchip_start = EM_design.set_variable('xchip_start', '42.82948mm')
    vec = [-(xoffset-xcorr)-xchip_start, #+offset_correction,
           0, 0]
    obj2_name = model.translate('cavity2', vec)

    # Chip setup
    cpl_width = EM_design.set_variable('cpl_width', f'{2.2:.3f}mm')
    l_chip = EM_design.set_variable('l_chip', '20mm')
    h_chip = EM_design.set_variable('h_chip', '5mm')
    t_chip = EM_design.set_variable('t_chip', '450um')
    chip_dims = [t_chip, l_chip+cpl_width, h_chip]

    # XXX: This is hard-coded, calculate this later
    xchip_start = EM_design.set_variable('xchip_start', '42.82948mm')
    chip_loc = [-xchip_start - xoffset/2,
                Stock_W/2 + l_chip / 2,
                Stock_H/2]

    # Edit sapphire material properties
    params = {'dielectric_loss_tangent' : 2e-8}
    project.edit_material(params, 'sapphire')
    chip = model.draw_box_center(chip_loc, chip_dims,
                                material='sapphire', name='chip')

    # Coupler cavity setup
    cpl_loc = [-xoffset/2 - xchip_start,
                Stock_W/2,
                Stock_H/2]
    cpl_dims = [1.5*xoffset, cpl_width, Stock_H/4]
    coupler = model.draw_box_center(cpl_loc, cpl_dims, name='coupler')

    # Subtractions and unions
    cavity_cpl = model.unite([obj_name, obj_name2, coupler])
    cavity_cpl = model.subtract(cavity_cpl, [chip], keep_originals=True)

## Pad & junction positions

In [7]:
# Junction and pads
# Planar pads
if overwrite:
    l_junc = EM_design.set_variable('l_junc', '50um')
    w_junc = EM_design.set_variable('w_junc', '10um')
    pad_w = 10 * w_junc
    pad_dims = [cpl_dims[1]/2 - l_junc/2, pad_w]

    pad1_loc = [-xchip_start - xoffset/2 - t_chip/2,
                Stock_W/2 - pad_dims[0]/2 - l_junc/2,
                Stock_H/2]
    pad2_loc = [-xchip_start - xoffset/2 - t_chip/2,
                Stock_W/2 + pad_dims[0]/2 + l_junc/2,
                Stock_H/2]

    # Pad geometry
    pad1 = model.draw_rect_center(pos=pad1_loc, x_size=0,
                              y_size=pad_dims[0],
                              z_size = pad_dims[1],
                              name='pad1')
    pad2 = model.draw_rect_center(pos=pad2_loc, x_size=0,
                              y_size=pad_dims[0],
                              z_size = pad_dims[1],
                              name='pad2')

    # Pad PEC assignment
    model.assign_perfect_E([pad1, pad2], name='junction_pads')

In [8]:
# Junction geometry, eigenmode
if overwrite:
    if solution_type == 'eigenmode':
        junc_loc = [-xchip_start - xoffset/2 - t_chip/2,
                    Stock_W/2,
                    Stock_H/2]
        jj=model.draw_rect_center(pos=junc_loc, x_size=0,
                                  y_size=l_junc,
                                  z_size = w_junc, name='jj')

        # Junction inductance assignment
        Lj_val = 0.1
        Cj = 94e-15; T1trans = 100e-6
        Rj_val = T1trans / Cj
        L_jj=EM_design.set_variable('Lj_1', '%.3fnH' % Lj_val)
        R_jj=EM_design.set_variable('Rj_1', '%.3fohm' % Rj_val)
        jj_line=model.draw_polyline(jj.make_center_line(axis='Y'), 
                                        closed=False, name='jj_line')
        jj.make_rlc_boundary(l=L_jj, r=R_jj, name='JJ_LJ', axis='Y')

        EM_setup=EM_design.create_em_setup(name='Test_EM', 
                                   min_freq_ghz=2, 
                                   n_modes=8, 
                                   max_delta_f=1.0, 
                                   min_converged=1, 
                                   converge_on_real=True)

# time.sleep(5)

In [9]:
#setup the analysis and optimetrics
opti_setup = HFSS.Optimetrics(EM_design)

#set the optimetrics step size and number of steps
step = .2
num_step = 1

sweep_type = 'decade'

#create optimetrics setup
swp_var = L_jj
swp_var_unit = 'nH'
#swp_var = xoffset
#swp_var_unit = 'mm'
start = 0.1
stop = 100
dstep = 2
num_step = 3 # int(np.round((stop-start)/dstep) + 1)

tstamp = datetime.datetime.today().strftime('%y%m%d_%H_%M_%S')
if sweep_type == 'decade':
    opti_setup.create_setup(variable=swp_var,
                        swp_type= 'decade',
                        swp_params=('%.2f%s'%(start, swp_var_unit),
                                    '%.2f%s'%(stop, swp_var_unit), 
                                    str(num_step)),
                        name=f'{swp_var}_Sweep_{tstamp}',
                        save_fields=True,
                        solve_with_copied_mesh_only=False,
                        copy_mesh=False)
elif sweep_type == 'linear':
    opti_setup.create_setup(variable=swp_var,
                        swp_type= 'linear_step',
                        swp_params=('%.2f%s'%(start, swp_var_unit),
                                    '%.2f%s'%(stop, swp_var_unit), 
                                    '%.2f%s' \
                                    % (((stop - start) / num_step),
                                       swp_var_unit)),
                        name=f'{swp_var}_Sweep_{tstamp}',
                        save_fields=True,
                        solve_with_copied_mesh_only=False,
                        copy_mesh=False)
else:
    raise ValueError(f'sweep_type ({sweep_type}) unrecognized.')
            
#solve it
opti_setup.solve_setup(f'{swp_var}_Sweep_{tstamp}')

#save project
project.save()

NameError: name 'L_jj' is not defined

In [2]:
def set_axes_fonts(ax, fsize):
    """
    Set axes font sizes because it should be abstracted away
    """
    for tick in ax.get_xticklabels():
        tick.set_fontsize(fsize)
    for tick in ax.get_yticklabels():
        tick.set_fontsize(fsize)

def set_xaxis_rot(ax, angle=45):
    """
    Rotate the x-axis labels
    """
        
    for tick in ax.get_xticklabels():
        tick.set_rotation(angle)

def set_leg_outside(ax, fsize):
    """
    Sets the legend location outside
    """
    
    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    # Put a legend to the right of the current axis
    hdls, legs = ax.get_legend_handles_labels()
    leg = ax.legend(hdls, legs, loc='center left', bbox_to_anchor=(1, 0.5), \
                    fontsize=fsize, framealpha=0.)

    return leg

In [3]:
def plot_2d_cmap(x, y, z, fname,                                                
                 xstr='', ystr='',                                              
                 tstr='', cbar_str='',                                          
                 cmap=cm.inferno,
                 xyscales={'x' : 'linear', 'y': 'linear'},
                 interp_zeros=False,
                interp_std_dev=10,
                image_rot_angle=0,
                norm_type='linear',
                zref=None, 
                fsize=30,
                tsize=36):
    """                                                                         
    Plot 2D colormap data such that                                             
                                                                                
         -----------------------                                                
         |                     |                                                
         |                     |                                                
    y    |          z          |                                                
         |                     |                                                
         |                     |                                                
         -----------------------                                                
                                                                                
                    x                                                           
                                                                                
    Parameters:                                                                 
    ----------                                                                  
                                                                                
    x, y:       independent variables                                           
    z:          resulting data, z = z(x, y)                                     
    fname:      output figure filename, relative path with file extension       
    xstr:       x-axis label                                                    
    ystr:       y-axis label                                                    
    tstr:       title label                                                     
    cbar_str:   lable for color bar                                             
    cmap:       colormap to use on the 2D figure output                         
                                                                                
    """
    # Interpolate the data
    if interp_zeros:
        hot_pixels, z = fix_outlier_pixels(z, tolerance=interp_std_dev)
    
    # Setup the color map, normalizations, etc
    if norm_type == 'linear':
        norm = mpl.colors.Normalize(z.min(), z.max())
    elif norm_type == 'log':
        norm = mpl.colors.LogNorm(z.min(), z.max())
    elif (norm_type == 'ref') and zref:
        norm = mpl.colors.Normalize(min(zref), max(zref))
    else:
        raise ValueError(f'norm_type ({norm_type}) not recognized')                                         
                                                                                
    # Setup the figure and axes                                                 
    fig, ax = plt.subplots(1, 1, figsize=(10, 8), tight_layout=True)
    
    # Set the scale bar and levels
    if (norm_type == 'ref') and zref:
        levels = MaxNLocator(nbins=20).tick_values(min(zref), max(zref))
    else:
        levels = MaxNLocator(nbins=20).tick_values(z.min(), z.max())
    if image_rot_angle:
        z = scipy.ndimage.rotate(z, image_rot_angle, reshape=True)
    #plt1 = ax.imshow(z, cmap=cmap, norm=norm)
    plt1 = ax.pcolormesh(x, y, z, cmap=cmap, norm=norm)
    ax.set_xlabel(xstr, fontsize=fsize)                                         
    ax.set_ylabel(ystr, fontsize=fsize)                                         
    ax.set_title(tstr, fontsize=tsize)
    
    # Rotate the xlabels. Set both x and y labels to headers[1:]
    ax.set_xscale(xyscales['x'])
    ax.set_yscale(xyscales['y'])
    # set_xaxis_rot(ax, 45)
                                                                                
    # Set the axis tick labels to a reasonable size                             
    set_axes_fonts(ax, fsize)                                                   
                                                                                
    # Set the color bar, offset the title slightly from top                     
    cbar = fig.colorbar(plt1, ax=ax)                                            
    cbar.ax.set_title(cbar_str, fontsize=fsize, y=1.025)                        
    cbar.ax.tick_params(labelsize=fsize)                                        
                                                                                
    # Write the results to file
    print(f'Writing {fname} ...')
    fig.savefig(fname, format='pdf', transparent=True)                  

## Read the electric field data to perform the integrals

In [13]:
# Setup the pyEPR project
Nmodes = 6
pinfo = pyEPR.ProjectInfo(HFSS_path+'\\HFSS',  
                         project_name,
                         design_name)
# Setup for EPR anaylsis
eprh = pyEPR.DistributedAnalysis(pinfo)

# Read off the inductances and x-offsets from the variations
variations = eprh.solutions.list_variations()
Ljstrs = []; Lj_vals = []
#xoffstrs = []; xoff_vals = []
for v in variations:
    Ljstrs.append(regex.findall('Lj_1=\'(.*?nH)', v)[0].replace('.', 'p'))
    Lj_vals.append(float(regex.findall('Lj_1=\'(.*?)nH', v)[0]))
 #   xoffstrs.append(regex.findall('x_offset=\'(.*?mm)', v)[0].replace('.', 'p'))
 #   xoff_vals.append(regex.findall('x_offset=\'(.*?mm)', v)[0])

INFO 12:32PM [connect]: Connecting to Ansys Desktop API...
INFO 12:32PM [load_ansys_project]: 	File path to HFSS project found.
INFO 12:32PM [load_ansys_project]: 	Opened Ansys App
INFO 12:32PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 12:32PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    Z:/tcav/ipynbs/HFSS/
	Project:   planar_pads_l_coupled_three_cavities_degen_eigen_230829
INFO 12:32PM [connect]: 	Opened active design
	Design:    planar_pads_l_coupled_three_cavities_degen_eigen_hfss_design_230829 [Solution type: Eigenmode]
INFO 12:32PM [get_setup]: 	Opened setup `Test_EM`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 12:32PM [connect]: 	Connection to Ansys established successfully. 😀 

ERROR 12:32PM [setup_data]: WARNING!   DESIGN FILENAME MAY BE TOO LONG! 


Design "planar_pads_l_coupled_three_cavities_degen_eigen_hfss_design_230829" info:
	# eigenmodes    8
	# variations    0


In [14]:
Lj_sort_idxs = np.argsort(Lj_vals)
Lj_vals = np.unique(np.asarray(Lj_vals)[Lj_sort_idxs])
Ljstrs = np.asarray(Ljstrs)[Lj_sort_idxs]

In [15]:
Lj_sort_idxs

array([], dtype=int64)

In [16]:
Lj_vals

array([], dtype=float64)

In [17]:
Ljstrs

array([], dtype=float64)

In [17]:
vstrs = [f'LJ{l}' for l in Ljstrs]

In [18]:
vstrs

['LJ0p10000000000000001nH',
 'LJ0p215443469003188nH',
 'LJ0p46415888336127797nH',
 'LJ1nH',
 'LJ2p1544346900318798nH',
 'LJ4p6415888336127802nH',
 'LJ10nH',
 'LJ21p5443469003188nH',
 'LJ46p4158883361278nH',
 'LJ100nH']

In [19]:
vars_sorted = np.asarray(eprh.variations)[Lj_sort_idxs]

In [20]:
[eprh._get_lv(v) for v in vars_sorted]

[['cpl_width:=',
  '2.2000000000000002mm',
  'h_chip:=',
  '5mm',
  'l_chip:=',
  '20mm',
  'l_junc:=',
  '50um',
  'Lj_1:=',
  '0.10000000000000001nH',
  'Rj_1:=',
  '1063829787.234ohm',
  'Stock_H:=',
  '2.25in',
  'Stock_H2:=',
  '2.25in',
  'Stock_L:=',
  '-2in',
  'Stock_L2:=',
  '-1.75in',
  'Stock_W:=',
  '0.625in',
  'Stock_W2:=',
  '0.625in',
  't_chip:=',
  '450um',
  'w_junc:=',
  '10um',
  'x_offset:=',
  '15mm',
  'xchip_start:=',
  '42.829479999999997mm',
  'xoff_corr:=',
  '8.9968724630000008mm'],
 ['cpl_width:=',
  '2.2000000000000002mm',
  'h_chip:=',
  '5mm',
  'l_chip:=',
  '20mm',
  'l_junc:=',
  '50um',
  'Lj_1:=',
  '0.215443469003188nH',
  'Rj_1:=',
  '1063829787.234ohm',
  'Stock_H:=',
  '2.25in',
  'Stock_H2:=',
  '2.25in',
  'Stock_L:=',
  '-2in',
  'Stock_L2:=',
  '-1.75in',
  'Stock_W:=',
  '0.625in',
  'Stock_W2:=',
  '0.625in',
  't_chip:=',
  '450um',
  'w_junc:=',
  '10um',
  'x_offset:=',
  '15mm',
  'xchip_start:=',
  '42.829479999999997mm',
  'xoff_co

## Need to manually create Vector_H plot in GUI first
* TODO: Create a new field plot, then export the field data with ansys.py

In [ ]:
# Build a string for each variation to label the output files
Nmodes = 4
for idx, v in enumerate(vars_sorted):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Variation: {vstrs[idx]}')
    for kp in range(Nmodes):
        # Set the solution to mode kp
        eprh.solutions.set_mode(kp+1)
        # time.sleep(0.5)
        print(f'Mode-{kp+1}')
        des = EM_design._design
        omod = des.GetModule('FieldsReporter')
        omod.ExportFieldPlot("Vector_H1", False, f"Z:\\tcav\\data\\hfields_mode_remesh2_{kp+1}_{vstrs[idx]}.aedtplt")
print('Finished writing field data to file.')

## Need to manually create Vector_E plot in GUI first

In [ ]:
# Build a string for each variation to label the output files
Nmodes = 4
for idx, v in enumerate(vars_sorted):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Variation: {vstrs[idx]}')
    for kp in range(Nmodes):
        # Set the solution to mode kp
        eprh.solutions.set_mode(kp+1)
        # time.sleep(0.5)
        print(f'Mode-{kp+1}')
        des = EM_design._design
        omod = des.GetModule('FieldsReporter')
        omod.ExportFieldPlot("Vector_E1", False, f"Z:\\tcav\\data\\efields_mode_remesh2_{kp+1}_{vstrs[idx]}.aedtplt")
print('Finished writing field data to file.')

## Call the integral code with the participations included

In [523]:
from importlib import reload
reload(fint)
Lj_vals = np.asarray([#0.1, 1., 10., 100.])
    1., 10., 100.])
Lj_strs = ['LJ0p10000000000000001nH',
 'LJ0p215443469003188nH',
 'LJ0p46415888336127797nH',
 'LJ1nH',
 'LJ2p1544346900318798nH',
 'LJ4p6415888336127802nH',
 'LJ10nH',
 'LJ21p5443469003188nH',
 'LJ46p4158883361278nH',
 'LJ100nH']
Lj_strs = [#'LJ0p10000000000000001nH',
  'LJ1nH',
 'LJ10nH',
 'LJ100nH']

data_path = f'../data'
hfield_format = f'hfields_mode_remesh2_%d_%s.aedtplt'
efield_format = f'efields_mode_remesh2_%d_%s.aedtplt'
Nmodes = 2
myfields = fint.FieldIntegrals(# eprh,
                 Nmodes, data_path,
                 Lj_vals, np.array([2e-15]*len(Lj_vals)),
                 Lj_strs,
                 hfield_format, efield_format,
                 jjstr='jj_line', LJstr='Lj_1',
                 jj_area = (50e-6)**2, debug=False,
                normalize_fields=True)

Integrals will use normalized fields.


In [524]:
AE, AH, XE, XH = myfields.solve_AB_matrix_eqs()

load_fields() modes: [1 2]
Loading Mode-1 ...
Loading Mode-2 ...
Computing Akkp ...
Running kkp1_1_LJLJp_LJ1nH_LJ1nH ...
Running kkp1_1_LJLJp_LJ1nH_LJ10nH ...
Running kkp1_1_LJLJp_LJ1nH_LJ100nH ...
Running kkp1_1_LJLJp_LJ10nH_LJ1nH ...
Running kkp1_1_LJLJp_LJ10nH_LJ10nH ...
Running kkp1_1_LJLJp_LJ10nH_LJ100nH ...
Running kkp1_1_LJLJp_LJ100nH_LJ1nH ...
Running kkp1_1_LJLJp_LJ100nH_LJ10nH ...
Running kkp1_1_LJLJp_LJ100nH_LJ100nH ...
Running kkp1_2_LJLJp_LJ1nH_LJ1nH ...
Running kkp1_2_LJLJp_LJ1nH_LJ10nH ...
Running kkp1_2_LJLJp_LJ1nH_LJ100nH ...
Running kkp1_2_LJLJp_LJ10nH_LJ1nH ...
Running kkp1_2_LJLJp_LJ10nH_LJ10nH ...
Running kkp1_2_LJLJp_LJ10nH_LJ100nH ...
Running kkp1_2_LJLJp_LJ100nH_LJ1nH ...
Running kkp1_2_LJLJp_LJ100nH_LJ10nH ...
Running kkp1_2_LJLJp_LJ100nH_LJ100nH ...
Running kkp2_1_LJLJp_LJ1nH_LJ1nH ...
Running kkp2_1_LJLJp_LJ1nH_LJ10nH ...
Running kkp2_1_LJLJp_LJ1nH_LJ100nH ...
Running kkp2_1_LJLJp_LJ10nH_LJ1nH ...
Running kkp2_1_LJLJp_LJ10nH_LJ10nH ...
Running kkp2_1_LJLJp_LJ

In [546]:
AE[:,:,0,1]

array([[-0.  ,  0.29, -0.26],
       [ 0.27, -0.  , -0.23],
       [ 0.26,  0.25,  0.  ]])

In [547]:
AE[:,:,0,1]

array([[-0.  ,  0.29, -0.26],
       [ 0.27, -0.  , -0.23],
       [ 0.26,  0.25,  0.  ]])

### Compute the parametric coupling rates

In [558]:
from importlib import reload
reload(fint)
Lj_vals = np.asarray([# 0.1, 1., 10., 100.])
    1.,10.,100.])
Lj_strs = ['LJ0p10000000000000001nH',
 'LJ0p215443469003188nH',
 'LJ0p46415888336127797nH',
 'LJ1nH',
 'LJ2p1544346900318798nH',
 'LJ4p6415888336127802nH',
 'LJ10nH',
 'LJ21p5443469003188nH',
 'LJ46p4158883361278nH',
 'LJ100nH']
Lj_strs = [#'LJ0p10000000000000001nH',
  'LJ1nH',
 'LJ10nH',
 'LJ100nH']

data_path = f'../data'
hfield_format = f'hfields_mode_remesh2_%d_%s.aedtplt'
efield_format = f'efields_mode_remesh2_%d_%s.aedtplt'
Nmodes = 2
mypcpl = fint.ParametricCouplings(AE, AH, XE, XH)
np.set_printoptions(precision=2)

In [559]:
wk, gsmsk = mypcpl.get_w_gsms_k(0, units='Hz')

In [560]:
wk

array([[0.25, 0.03, 0.02],
       [0.02, 0.25, 0.02],
       [0.02, 0.02, 0.25]])

In [561]:
gsmsk

array([[ 3.99e-08, -5.14e-02, -4.94e-02],
       [-4.87e-02, -2.72e-07, -4.04e-02],
       [-4.89e-02, -4.28e-02, -6.99e-07]])

In [562]:
gbs, gtms = mypcpl.get_gbs_gtms_kkp(1, 0, units='Hz')

In [563]:
gbs

array([[-0.  ,  0.03,  0.04],
       [ 0.04, -0.  ,  0.03],
       [-0.04, -0.03,  0.  ]])

In [564]:
gtms

array([[-8.22e-05, -3.46e-02, -3.57e-02],
       [-3.55e-02,  3.67e-04, -3.02e-02],
       [ 3.57e-02,  2.94e-02, -6.29e-04]])

In [565]:
gbs, gtms = mypcpl.get_gbs_gtms_kkp(0, 1, units='Hz')

In [566]:
gbs

array([[-0.  ,  0.03,  0.04],
       [ 0.04, -0.  ,  0.03],
       [-0.04, -0.03,  0.  ]])

In [567]:
gtms

array([[-8.22e-05, -3.46e-02, -3.57e-02],
       [-3.55e-02,  3.67e-04, -3.02e-02],
       [ 3.57e-02,  2.94e-02, -6.29e-04]])

## Call the field integral code

In [ ]:
# Single point of entry inputs to class
Nmodes = 4
modes = np.linspace(1, Nmodes, Nmodes, dtype=int)
print(f'modes: {modes}')
# LJs = np.asarray([0.1, 1., 10., 100.])
# LJstrs = ['0p1', '1', '10']

# Instantiate the class object
data_path = f'../data'
myfields = fint.FieldIntegrals(Lj_vals, Ljstrs,
                               modes, data_path, debug=True,
                               has_updated_fields=True,
                               has_normalized_fields=False,
                               normalize_fields=True,
                               drop_max_field=1,
                               filename_format = f'hfields_mode_remesh2_%d_LJ%s.aedtplt',
                               timeout_limit=30.)
abscissa, efields = myfields.load_fields(return_fields=True)
keys = list(efields.keys())

In [ ]:
Bkkp = myfields.overlap_matrix_points(fname=None, cidxs=[3, 4],
                                      drop_points=1,#'../data/Akkp_nondegenerate_flutes_{dd}.csv',
                                      return_matrix=True)

In [ ]:
# Single point of entry inputs to class
Nmodes = 4
modes = np.linspace(1, Nmodes, Nmodes, dtype=int)
print(f'modes: {modes}')
# LJstrs = ['0p1', '1', '10']

# Instantiate the class object
data_path = f'../data'
myfields = fint.FieldIntegrals(Lj_vals, Ljstrs,
                               modes, data_path, debug=True,
                               has_updated_fields=True,
                               has_normalized_fields=False,
                               normalize_fields=True,
                               filename_format = f'efields_mode_remesh2_%d_LJ%s.aedtplt',
                               timeout_limit=30.)
abscissa, efields = myfields.load_fields(return_fields=True)
keys = list(efields.keys())

Akkp = myfields.overlap_matrix_points(fname=None, #'../data/Akkp_nondegenerate_flutes_{dd}.csv',
                               return_matrix=True)

In [ ]:
len(Akkp)

## Repair the $A_{34}$ and $A_{43}$ matrices

In [ ]:
Akkpcopy = Akkp.copy()
Bkkpcopy = Bkkp.copy()
A34 = np.zeros([len(Lj_vals), len(Lj_vals)])
A43 = np.zeros([len(Lj_vals), len(Lj_vals)])
B34 = np.zeros([len(Lj_vals), len(Lj_vals)])
B43 = np.zeros([len(Lj_vals), len(Lj_vals)])

for k1 in range(1, Nmodes+1):
    for k2 in range(1, Nmodes+1):
        for idx in range(len(Lj_vals)):
            for idx0 in range(len(Lj_vals)):
                if k1 != k2:
                    # Repair the Akkp
                    key34 = f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    key43 = f'kkp{k2}_{k1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    A34[idx, idx0] = Akkpcopy[key34]
                    A43[idx, idx0] = Akkpcopy[key43]
                    Akkp[key34] = (A34[idx, idx0] - A43[idx, idx0]) / 2
                    Akkp[key43] = (A43[idx, idx0] - A34[idx, idx0]) / 2
                    
                    # Repair the Bkkp
                    B34[idx, idx0] = Bkkpcopy[key34]
                    B43[idx, idx0] = Bkkpcopy[key43]
                    Bkkp[key34] = (B34[idx, idx0] - B43[idx, idx0]) / 2
                    Bkkp[key43] = (B43[idx, idx0] - B34[idx, idx0]) / 2

In [ ]:
Akkpcopy[key34]

In [ ]:
Akkpcopy['kkp3_4_LJLJp_1nH_10nH']

In [ ]:
Akkp['kkp3_4_LJLJp_1nH_10nH']

In [ ]:
Bkkp

In [ ]:
Akkp

In [ ]:
ww, qq = eprh.solutions.eigenmodes()

In [ ]:
ww

In [ ]:
qq

## Get the eigenmode frequencies and losses
* Note that frequencies are in GHz here

In [ ]:
wk = {}; Qk = {};
vstrs = [f'LJ{l}' for l in Ljstrs]
for idx, v in enumerate(eprh.variations):
    # Set the variation
    eprh.set_variation(v)
    lv = eprh._get_lv(v)
    print(f'Eigenmodes for variation {vstrs[idx]} ...')
    ww, qq = eprh.solutions.eigenmodes(lv=lv)
    key = f'{vstrs[idx]}'
    wk.update({key : ww})
    Qk.update({key : [ww1/qq1 for ww1, qq1 in zip(ww, qq)]})

In [ ]:
wk

In [ ]:
Qk

In [ ]:
LJ_dict = dict({k : v for k, v in zip(Ljstrs, Lj_vals)})

In [ ]:
LJ_dict

## Plot the modes vs. inductances

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
fsize = 20
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
idx_start = 4
Nmodes = 4
for k in range(Nmodes):
    wppp = np.array([wk[f'LJ{key}'][k] for key in Ljstrs])
    ax.plot(Lj_vals[2:], wppp[2:],
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
    #ax.plot(LJs/1e-9, [wp0[idx]/(2*np.pi*fscale)] * len(LJs),
    #        '--', color=cycle[idx%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Eigenfrequencies [GHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
#ax.set_ylim([5.88, 5.95])
set_axes_fonts(ax, fsize)
leg = set_leg_outside(ax, lsize)
ax.set_xscale('log')
fig.savefig(f'../figs/modes_hfields_bare_remesh2_nondegen_{dstr}.pdf', 
            format='pdf', bbox_extra_artists=(leg, ), bbox_inches='tight',
           transparent=True)

# Plot the eigenmode $T_1$ values

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale = 1e9
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
T1trans = 100
for k in range(Nmodes):
    Q = np.array([Qk[f'LJ{key}'][k] for key in Ljstrs])
    wppp = 2*np.pi*np.array([wk[f'LJ{key}'][k] for key in Ljstrs])*fscale
    T1nom = 1e6 * Q / wppp
    T1tot = T1nom * T1trans / (T1nom + T1trans)
    ax.plot(Lj_vals, T1nom,
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$T_1$ [$\mu$s]', fontsize=fsize)
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
set_leg_outside(ax, lsize)
#ax.set_ylim([1e-2, 350])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
ax.set_yscale('log')
fig.savefig(f'../figs/T1_modes_hfields_coupled_remesh2_eigenmode_{dstr}.pdf',
            format='pdf')

## Compute the $g_{kk'}$ and $\widetilde{\omega}_{k}$ from $A_{kk'}$

In [ ]:
"""
Compute the couplings from Akk, wk values
"""
# Iterate over all modes and inductances
gkk = {}
idx_start = 2; idx_stop = -1
Nmodes = 4
for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx, LJ in enumerate(Lj_vals):
            
            # Outside sqrt(wk wk') factor
            wkey = f'LJ{Ljstrs[idx]}'
            wkwkp = np.sqrt(wk[wkey][k] * wk[wkey][kp])
            
            for idx0, LJ0 in enumerate(Lj_vals):
                
                # Key to index gkk'
                gkey = f'kkp{k+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                Akkpsum = 0
                print(f'Computing g at {gkey}, wkwkp: {wkwkp:.2e} ...')
                for kpp in range(Nmodes):
                    key = f'kkp{kpp+1}_{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    keyp = f'kkp{kpp+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    Akppk = Akkp[key]
                    Akppkp = Akkp[keyp]
                    Akkpsum += Akppk * Akppkp

                # Store the results
                if (k != kp):
                    gkk.update({gkey : wkwkp * Akkpsum})

In [ ]:
'kkp1_1_LJLJp_0p10000000000000001nH_0p10000000000000001nH' in gkk.keys()

### Collect the results as a matrix

In [ ]:
gkk_matrix = np.zeros([Nmodes, Nmodes, len(Lj_vals), len(Lj_vals)])
for k in range(Nmodes):
    for kp in range(Nmodes):
        for idx, LJ in enumerate(Lj_vals):
            for idx0, LJ0 in enumerate(Lj_vals):
                if k != kp:
                    gkey = f'kkp{k+1}_{kp+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                    gkk_matrix[k, kp, idx, idx0] = gkk[gkey]


In [ ]:
gkk_matrix

In [ ]:
"""
Compute the couplings from Akk, wk values
"""
# Iterate over all modes and inductances
wkt = {}
idx_start = 2; idx_stop = -1
Nmodes = 4
for k in range(Nmodes):
    for idx, LJ in enumerate(Lj_vals):

        # Outside wk factor
        wkey = f'LJ{Ljstrs[idx]}'
        wkp = wk[wkey][k]

        for idx0, LJ0 in enumerate(Lj_vals):

            # Key to index gkk'
            wwkey = f'kkp{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
            Akkpsum = 0
            # print(f'Computing wkp at {wwkey}, wkwkp: {wkp:.2e} ...')
            for kpp in range(Nmodes):
                key = f'kkp{kpp+1}_{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
                Akppk = Akkp[key]
                Akkpsum += Akppk**2

            # Store the results
            wkt.update({wwkey : wkp * Akkpsum})

In [ ]:
len(wkt)

In [ ]:
wkt_matrix = np.zeros([Nmodes, len(Lj_vals), len(Lj_vals)])

## Compute the $\widetilde{\omega}_k(L_J, L_{J_0})$ matrix

In [ ]:
for k in range(Nmodes):
    for idx, LJ in enumerate(Lj_vals):
        for idx0, LJ0 in enumerate(Lj_vals):
            wwkkey = f'kkp{k+1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'
            wkt_matrix[k, idx, idx0] = wkt[wwkkey]

### Sort the frequencies for each $(L_J, L_{J_0})$ pair

In [ ]:
wkt_matrix_sorted = np.sort(wkt_matrix, axis=0)
wkt_sort_idx = np.argsort(wkt_matrix, axis=0)

In [ ]:
gkk_sort_idx = wkt_sort_idx[np.newaxis, :, :, :]

In [ ]:
gkk_sort_idx.shape

In [ ]:
gkk_matrix_sorted = gkk_matrix[gkk_sort_idx]

In [ ]:
gkk_matrix_sorted.shape

## Plot the sorted $\widetilde{\omega}_k(L_J, L_{J_0})$ values

In [ ]:
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)

for idx_fixed in range(len(Lj_vals)):
    print(f'LJ0-{idx_fixed} ...')
    fig, ax = plt.subplots(1, 1, tight_layout=True)
    for k in range(Nmodes):
        ax.plot(Lj_vals, wkt_matrix_sorted[k, :, idx_fixed],
            'o-', label=r'Mode-%d' % (k + 1),
            color=cycle[k%clen])
    ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
    ax.set_ylabel(r'$\widetilde{\omega}/2\pi$ [GHz]', fontsize=fsize)
    set_xaxis_rot(ax, 45)
    set_axes_fonts(ax, fsize)
    leg = set_leg_outside(ax, lsize)
    ax.set_xscale('log')
    fig.savefig(f'../figs/modes_eigenmode_remesh2_modified_LJ0_{Ljstrs[idx_fixed]}_{dstr}.pdf', 
                format='pdf', bbox_extra_artists=(leg, ), bbox_inches='tight',
               transparent=True)

In [ ]:
'kkp1_LJLJp_0p10000000000000001nH_56p234132519034901nH' in wkt.keys()

## Plot $g_{k_1,k_2}$ vs. $L_J, L_{J_0}$

In [ ]:
gmax = np.max(np.array(list(gkk.values()))) * 1e3
gmin = np.min(np.array(list(gkk.values()))) * 1e3

In [ ]:
gmax

In [ ]:
gmin

In [ ]:
#plot_2d_cmap(ydata/ymax,
#    zdata/zmax,
for k1 in [1,2,3,4]:
    for k2 in [1, 2, 3, 4]:
        if k1 != k2:
            gkk12 = np.array([[gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                               for idx in range(len(Lj_vals))] \
                               for idx0 in range(len(Lj_vals))])*1e3
            plot_2d_cmap(# LJs/1e-9,
                #Lj_vals[idx_start:idx_stop],
                #Lj_vals[idx_start:idx_stop],
                Lj_vals,
                Lj_vals,
                gkk12,
                # gkk_matrix_sorted[k1, k2, :, :],
                f'../figs/g{k1}_{k2}_hfields_remesh2_sorted_vs_LJLJ0_{dd}.pdf',
                xstr=r'$L_{J_0}$ [nH]',
                ystr=r'$L_{J}$ [nH]',
                tstr='',
                xyscales={'x' : 'log', 'y' : 'log'},
                cbar_str=r'$g_{%d%d}$ [MHz]' % (k1, k2),
                interp_zeros=False,
                interp_std_dev=3,
                image_rot_angle=0,
                norm_type='ref',
                zref=[gmin, gmax],
                fsize=40,
                tsize=46)

## Plot $A_{k_1,k_2}(L_{J}, L_{J_0})$ vs. $L_J, L_{J_0}$

In [ ]:
Amax = np.max(list(Akkp.values()))
Amin = np.min(list(Akkp.values()))

In [ ]:
Lj_vals

In [ ]:
Ljstrs

In [ ]:
Amax

In [ ]:
Amin

In [ ]:
for k1 in [1, 2, 3, 4]:
    for k2 in [1, 2, 3, 4]:

        Akk12 = np.array([[Akkp[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                           for idx in range(len(Lj_vals))] \
                           for idx0 in range(len(Lj_vals))])
        
        
        plot_2d_cmap(
            Lj_vals,
            Lj_vals,
            np.abs(Akk12),
            f'../figs/B{k1}_{k2}_mag_remesh2_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$|B_{%d%d}|$' % (k1, k2),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='ref',
            zref=[0, Amax],
            fsize=40,
            tsize=46)

In [ ]:
wmax = np.max(np.array(list(wkt.values())))
wmin = np.min(np.array(list(wkt.values())))

In [ ]:
wmax

In [ ]:
wmin

In [ ]:
#plot_2d_cmap(ydata/ymax,
#    zdata/zmax,
# for k1 in [1,2,3,4]:
for k1 in range(Nmodes):
        # wkk12 = np.array([[wkt[f'kkp{k1}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
        #                   for idx in range(len(Lj_vals))] \
        #                   for idx0 in range(len(Lj_vals))])
        plot_2d_cmap(# LJs/1e-9,
            Lj_vals,
            Lj_vals,
            #wkk12,
            wkt_matrix_sorted[k1, :, :],
            # f'../figs/w_{k1}_vs_LJLJ0_{dd}.pdf',
            f'../figs/w_{k1+1}_hfields_remesh2_sorted_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$\omega_{%d}$ [GHz]' % (k1),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='ref',
            zref=[wmin, wmax],
            fsize=40,
            tsize=46)

## Compute the derivatives to estimate the driven coupling strengths

In [ ]:
dLJ = 1 # 10 nH as a possible driving point
for k1 in [1,2,3,4]:
    for k2 in [1, 2, 3, 4]:
        if k1 != k2:
            gkk12 = np.array([[gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                               for idx in range(len(Lj_vals))] \
                               for idx0 in range(len(Lj_vals))])
            gkk12_grad = np.array([np.gradient(gkk12[:, j], Lj_vals) for j in range(len(Lj_vals))])
            epskk = gkk12_grad * dLJ * 1e3
            # Write the results to file
            plot_2d_cmap(# LJs/1e-9,
            Lj_vals,
            Lj_vals,
            epskk,
            f'../figs/eps_{k1}_{k2}_hfields_remesh2_vs_LJLJ0_{dd}.pdf',
            xstr=r'$L_{J_0}$ [nH]',
            ystr=r'$L_{J}$ [nH]',
            tstr='',
            xyscales={'x' : 'log', 'y' : 'log'},
            cbar_str=r'$\epsilon_{g_{%d %d}}$ [MHz]' % (k1, k2),
            interp_zeros=False,
            interp_std_dev=3,
            image_rot_angle=0,
            norm_type='ref',
            zref=[gmin, gmax],
            fsize=40,
            tsize=46)

## Plot $\epsilon_{g_{12}}$ for a fixed $L_{J_0}$

In [ ]:
LJidx0 = 0
dLJ = 10 # 10 nH as a possible driving point
k1 = 1; k2 = 2

fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
fsize = 20
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
idx_start = 4
cidx = 0
for idx0 in [4, 8, 10, 12]:
    gkk12 = np.array([gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[LJidx0]}'] \
                               for idx in range(len(Lj_vals))])
    gkk12_grad = np.gradient(gkk12, Lj_vals)
    epskk = gkk12_grad * Lj_vals[idx0] * 1e3
    ax.plot(Lj_vals, epskk / 2,
        'o-', color=cycle[cidx%clen],
           label=r'$\delta L_{J}$ = %.2f nH' % Lj_vals[idx0])
    cidx += 1

ax.axhline(y=0, color='k', ls='--')
ax.set_title(r'$L_{J_0} = %.2f$ nH' % Lj_vals[LJidx0],
             fontsize=fsize)
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$\frac{1}{2}\epsilon_{g_{%d%d}}$ [MHz]' % (k1, k2), fontsize=fsize)
set_xaxis_rot(ax, 45)
#ax.set_ylim([-0.5, 1.5])
set_axes_fonts(ax, fsize)
ax.set_xscale('log')
leg = set_leg_outside(ax, lsize)
fig.savefig(f'../figs/epsg_{k1}{k2}_hfields_bbq_1_10_100nH_remesh2_{dstr}.pdf', 
            bbox_extra_artists=(leg, ), bbox_inches='tight',
            format='pdf', transparent=True)

## Combine the geometric mean and field analysis data

In [ ]:
def load_ydata_from_txt(filenames, delimiter=','):
    fdata = np.array([])
    ydata = np.array([], dtype=complex)
    # Iterate over all discrete frequency sweeps
    for fname in filenames:
        print(f'Reading {fname} ...')
        data = np.genfromtxt(fname, delimiter=delimiter).T
        f   = data[0]
        yre = data[1]
        yim = data[2]

        fdata = np.hstack((fdata, f))
        ydata = np.hstack((ydata, yre + 1j*yim))
    return fdata, ydata

In [ ]:
filenames = [f'../data/ydata_230110_sweep_{i}.csv' for i in range(21)]
fdata1, ydata1 = load_ydata_from_txt(filenames)
filenames = [f'../data/ydata_230210_sweep_{i}.csv' for i in range(21)]
fdata2, ydata2 = load_ydata_from_txt(filenames)

In [ ]:
def get_wp_Cp_Lp(f, y, LJ0=10e-9, CJ0=2e-15, fscale=1e9, use_Zq=False,
                GJ0=7.748e-10):
    """
    Converts frequency-dependent imaginary parts
    of the admittance data to poles,
    capacitances, and inductances
    """
    # Add the junction inductance and capacitance to the
    # total admittance
    # Compute the zero crossings
    fcpy = np.copy(f * fscale)
    ycpy = np.copy(np.complex128(y))
    if LJ0:
        ycpy += (1. / (1j * fcpy * 2 * np.pi * LJ0)) \
          + 1j * fcpy * 2 * np.pi * CJ0
    if LJ0 and GJ0:
        ycpy += (1. / (1j * fcpy * 2 * np.pi * LJ0)) \
             + 1j * fcpy * 2 * np.pi * CJ0 \
             + GJ0
    yim = ycpy.imag
    ypp = np.gradient(ycpy, 2*np.pi*(fcpy[1]-fcpy[0]))
    zero_xing_idxs = np.where(np.diff(np.sign(yim)) and ypp.imag > 0)[0]
    wp = 2 * np.pi * fcpy[zero_xing_idxs]
    yp = yim[zero_xing_idxs]

    # Compute the capacitances from the derivatives
    Cp = 0.5 * np.abs(ypp.imag[zero_xing_idxs])
    Lp = 1. / (wp**2 * Cp)
    Zp = np.sqrt(Lp / Cp)
    Rp = 1. / np.abs(np.real(ycpy[zero_xing_idxs]))
    Qp = wp * Rp * Cp
    
    # Physical constants
    h    = 6.62607015e-34 
    hbar = h / (2 * np.pi)
    e    = 1.602176634e-19
    Phi0 = h / (2 * e)
    
    # Compute Ej, Ec
    if LJ0 and CJ0:
        # Compute the Josephson inductive and charging energies
        Ej = (Phi0 / (2 * np.pi))**2 / LJ0
        Ec = e**2 / (2 * CJ0)

        # Compute inductive energies, EL
        EL = 0.5 * Phi0**2 / Lp

        # Compute the phase zero point fluctuations
        GQ = 2 * e**2 / h
        phip2 = 2 * np.pi * GQ * Zp

        # Compute the participations
        # p = phi2 * 2 * Ej / (hbar * wp)
        pl = Lp / LJ0
        pc = Cp / CJ0

        # Compute the gs / Ds
        Zq = np.sqrt(LJ0 / CJ0)
        gD = np.sqrt(Zp / Zq)

        # Compute the g's in units of Hz, divide by h to get v, Ej = hv
        gpq = (2 * np.pi * GQ * Ej / 24.) * sum(phip2) * np.asarray([[
            np.sqrt(Zpp * Zqq) / h for p, Zpp in enumerate(Zp)]
                               for q, Zqq in enumerate(Zp) ])
        gpq = gpq.reshape([len(gpq), len(gpq)])
        
        # Compute the chis, anharmonicities
        # In units of Hz
        chipp  = -Ec * gD**2 / h
        chippp = np.asarray([-2 * np.sqrt(chipp[p] * chipp[q]) 
                             for p in range(len(chipp))
                             for q in range(len(chipp))])
        chippp = chippp.reshape([len(chipp), len(chipp)])

        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, pl, pc, gpq, chippp
    
    else:
        return wp, Lp, Cp, Qp, fcpy, ycpy, yp, None, None, None, None

In [ ]:
def get_bbq_params(fdata, ydata, NLJ=25, Nmodes=6):
    wps = []
    pcs = []
    pls = []
    gDs = []
    Cps = []
    Lps = []
    Qps = []
    chis = []

    # Whether or not to use the largest impedance mode
    # as the junction mode in the calculations
    use_Zq = False
    print_results = True

    LJs = np.logspace(np.log10(0.1e-9), np.log10(1000e-9), NLJ)
    LJs = np.logspace(np.log10(0.1e-9), np.log10(100e-9), NLJ)
    LJ0 = 10e-9
    #for idx, x in enumerate(xdata):
    for LJ in LJs:
        wpp, Lp, Cp, Qp, _, _, _, pl, pc, gD, chip \
        = get_wp_Cp_Lp(fdata, ydata, LJ0=LJ, fscale=fscale, use_Zq=use_Zq)
        wps.append(np.sort(wpp[0:Nmodes]))
        #wps.append(wpp[0:Nmodes])
        pls.append(pl[0:Nmodes])
        pcs.append(pc[0:Nmodes])
        gDs.append(gD[0:Nmodes, 0:Nmodes])
        Lps.append(Lp[0:Nmodes])
        Cps.append(Cp[0:Nmodes])
        Qps.append(Qp[0:Nmodes])
        chis.append(chip[0:Nmodes, 0:Nmodes])
        if print_results:
            print(f'wps: {wps}')
        # wps.append(wpp)
        # print(f'LJ: {LJ}, wpp: {wpp}')
    wps = np.asarray(wps).T
    pls  = np.asarray(pls).T
    pcs  = np.asarray(pls).T
    gDs = np.asarray(gDs).T
    Lps = np.asarray(Lps).T
    Cps = np.asarray(Cps).T
    Qps = np.asarray(Qps).T
    chis = np.asarray(chis).T

    return wps, pls, pcs, gDs, Lps, Cps, Qps, chis

In [ ]:
wps1, pls1, pcs1, gDs1, Lps1, Cps1, Qps1, chis1 = get_bbq_params(fdata1, ydata1)
wps2, pls2, pcs2, gDs2, Lps2, Cps2, Qps2, chis2 = get_bbq_params(fdata2, ydata2)

In [ ]:
fig, ax = plt.subplots(1, 1, tight_layout=True)
lsize = 14
fscale2 = 1e6
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
split1 = (wps1[1]-wps1[0])/(2*np.pi*fscale2)
split2 = (wps2[1]-wps2[0])/(2*np.pi*fscale2)
NLJ = 25
LJs = np.logspace(np.log10(0.1e-9), np.log10(100e-9), NLJ)
ax.plot(LJs/1e-9, split1,
           'o-',
            color=cycle[0],
           label=r'$g_{\mathrm{a}}$')
ax.plot(LJs/1e-9, split2,
           'o-',
            color=cycle[1],
           label=r'$g_{\mathrm{b}}$')
ax.plot(LJs/1e-9, np.sqrt(split1 * split2),
           'o-',
            color=cycle[2],
           label=r'$g_{\mathrm{ab}}$')
# ax.plot(LJs/1e-9, np.ones(LJs.size) * dw/(fscale2), 'k--')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'Mode Splitting [MHz]', fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
leg = set_leg_outside(ax, 16)
#ax.set_ylim([0, 150])
#ax.set_xlim([0.5, 2.2])
ax.set_xscale('log')
fig.savefig(f'../figs/modes_ab_splitting_term_bbq_inductive_planar_sapphire_{dstr}.pdf',
            transparent=True,
            bbox_extra_artists=(leg, ), bbox_inches='tight',
            format='pdf')

In [ ]:
LJidx0 = 0
k1 = 1; k2 = 2

fig, ax = plt.subplots(1, 1, tight_layout=True)
dstr = datetime.datetime.today().strftime('%y%m%d')
fsize = 20
lsize = 14
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
clen = len(cycle)
idx_start = 4
cidx = 0
for idx0 in [4, 8, 10, 12]: #, 16, 20, 22]: #, 11, 12, 19, 20, 22, 24]:
    gkk12 = np.array([gkk[f'kkp{k1}_{k2}_LJLJp_{Ljstrs[idx]}_{Ljstrs[idx0]}'] \
                               for idx in range(len(Lj_vals))])    
    ax.plot(Lj_vals, (gkk12) * 1e3,
        'o-', color=cycle[cidx%clen],
           label=r'$L_{J_0}$ = %.2f nH' % Lj_vals[idx0])
    cidx += 1
ax.plot(LJs/1e-9, np.sqrt(split1 * split2), ls='-.', lw=2,
        color='gray',
        label=r'$g_{ab}=\sqrt{g_{aa}g_{bb}}$')
ax.axhline(y=0, color='k', ls='--')
# ax.axhline(y=22, color='k', ls='--')
ax.set_xlabel(r'$L_J$ [nH]', fontsize=fsize)
ax.set_ylabel(r'$g_{%d%d}$ [MHz]' % (k1, k2), fontsize=fsize)
set_xaxis_rot(ax, 45)
set_axes_fonts(ax, fsize)
ax.set_xscale('log')
# ax.set_yscale('log')
leg = set_leg_outside(ax, lsize)
fig.savefig(f'../figs/g_bfields_bbq_1_10_100nH_remesh2_{dstr}.pdf', 
            bbox_extra_artists=(leg, ), bbox_inches='tight',
            format='pdf', transparent=True)